## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
import pandas as pd
import requests
import gmaps

from config import g_key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,2021-02-26 00:02:06,-34.4187,19.2345,63.00,85,43,12.91,scattered clouds
1,1,Bluff,NZ,2021-02-26 00:02:32,-46.6000,168.3333,64.00,69,96,4.00,overcast clouds
2,2,Puerto Ayora,EC,2021-02-26 00:03:36,-0.7393,-90.3518,80.01,90,76,5.01,light rain
3,3,Sao Miguel Do Araguaia,BR,2021-02-26 00:05:49,-13.2750,-50.1628,74.64,88,52,3.44,broken clouds
4,4,Dajal,PK,2021-02-26 00:05:50,29.5577,70.3762,64.69,29,0,5.06,clear sky


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# 4a. Determine if there are any empty rows.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Sao Miguel Do Araguaia,BR,74.64,broken clouds,-13.2750,-50.1628,
13,Barabai,ID,78.85,overcast clouds,-2.5833,115.3833,
14,Rikitea,PF,75.56,moderate rain,-23.1203,-134.9692,
16,Thai Binh,VN,71.60,overcast clouds,20.4500,106.3333,
19,Mar Del Plata,AR,71.01,clear sky,-38.0023,-57.5575,
22,Kapaa,US,75.20,light rain,22.0752,-159.3190,
24,Saint-Philippe,RE,78.80,overcast clouds,-21.3585,55.7679,
25,Pisco,PE,73.40,clear sky,-13.7000,-76.2167,
32,Seka,TH,78.01,broken clouds,17.9285,103.9552,
33,Galle,LK,75.20,few clouds,6.0367,80.2170,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
# 6b. Iterate through the hotel DataFrame.
    # 6c. Get latitude and longitude from DataFrame    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # 6e. Make request and retrieve the JSON data from the search. 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        continue
        

In [23]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [25]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv

output_data_file = "WeatherPy_vacation.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
# 11b. Display the figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))